# Chapter 21: Logging Fundamentals

The `logging` module is Python's built-in framework for emitting structured, leveled log
messages. It provides far more control and flexibility than `print()` statements, and is
the standard way to record what your program is doing at runtime.

## Topics Covered
- **Why logging over print()**: Advantages of structured logging
- **Log levels**: DEBUG, INFO, WARNING, ERROR, CRITICAL with numeric values
- **Loggers**: Creating loggers with `getLogger()`, setting levels
- **Handlers**: `StreamHandler` and basic handler setup
- **Formatters**: Format strings and common placeholders
- **basicConfig()**: Quick setup for simple use cases
- **LogRecord attributes**: How log records flow through the system
- **Practical**: Setting up logging for an application module

## Why Logging Over print()

Using `print()` for debugging and monitoring has significant limitations:

| Feature | `print()` | `logging` |
|---------|-----------|----------|
| Severity levels | No | DEBUG, INFO, WARNING, ERROR, CRITICAL |
| Output destination | stdout only | Console, files, network, email, etc. |
| Enable/disable | Must remove/comment code | Change level or disable handler |
| Timestamps | Manual | Built-in with `%(asctime)s` |
| Source tracking | Manual | Automatic module/line number |
| Thread safety | No | Yes |
| Configuration | Hardcoded | Runtime-configurable |

The `logging` module lets you leave diagnostic code in your program permanently and control
its verbosity through configuration rather than code changes.

In [ ]:
# The problem with print() debugging
def calculate_discount(price: float, discount_pct: float) -> float:
    """Calculate discounted price -- print version."""
    print(f"DEBUG: price={price}, discount_pct={discount_pct}")  # Must remove later
    if discount_pct < 0 or discount_pct > 100:
        print(f"WARNING: Invalid discount {discount_pct}")       # No severity level
        return price
    result = price * (1 - discount_pct / 100)
    print(f"DEBUG: result={result}")                             # Clutters output
    return result


print(calculate_discount(100.0, 20.0))
print(calculate_discount(50.0, -5.0))

## Log Levels: Severity Classification

The logging module defines five standard levels, each with a numeric value. Messages are
only emitted if their level meets or exceeds the logger's configured threshold.

| Level | Numeric Value | Purpose |
|-------|--------------|--------|
| `DEBUG` | 10 | Detailed diagnostic information |
| `INFO` | 20 | Confirmation that things are working |
| `WARNING` | 30 | Something unexpected but not an error (default level) |
| `ERROR` | 40 | A serious problem; some functionality failed |
| `CRITICAL` | 50 | A very serious error; the program may not continue |

In [ ]:
import logging

# Numeric values of each level
levels: list[tuple[str, int]] = [
    ("DEBUG", logging.DEBUG),
    ("INFO", logging.INFO),
    ("WARNING", logging.WARNING),
    ("ERROR", logging.ERROR),
    ("CRITICAL", logging.CRITICAL),
]

for name, value in levels:
    print(f"{name:>10} = {value}")

# You can also look up level names and values
print(f"\nLevel name for 30: {logging.getLevelName(30)}")
print(f"Level value for 'ERROR': {logging.getLevelName('ERROR')}")

## Creating Loggers with getLogger()

`logging.getLogger(name)` returns a logger with the given name, creating it if necessary.
Calling it again with the same name always returns the **same** logger instance (singleton
pattern). The convention is to use `__name__` as the logger name, which gives you the
module's fully qualified name.

The **root logger** is obtained by calling `getLogger()` with no arguments. All other loggers
are descendants of the root logger.

In [ ]:
import logging

# Create a named logger (convention: use __name__)
logger = logging.getLogger("myapp.utils")

# getLogger with the same name returns the same instance
same_logger = logging.getLogger("myapp.utils")
print(f"Same instance: {logger is same_logger}")  # True

# The root logger
root = logging.getLogger()
print(f"Root logger name: {root.name!r}")
print(f"Root logger level: {root.level} ({logging.getLevelName(root.level)})")

# Default level for new loggers is NOTSET (0), which defers to parent
print(f"\nLogger name: {logger.name!r}")
print(f"Logger level: {logger.level} ({logging.getLevelName(logger.level)})")
print(f"Effective level: {logger.getEffectiveLevel()} "
      f"({logging.getLevelName(logger.getEffectiveLevel())})")

In [ ]:
import logging

# Setting the level on a logger controls which messages pass through
logger = logging.getLogger("myapp.demo")
logger.setLevel(logging.DEBUG)

# We need a handler to see output -- add a StreamHandler
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)

# All five levels
logger.debug("This is a DEBUG message")
logger.info("This is an INFO message")
logger.warning("This is a WARNING message")
logger.error("This is an ERROR message")
logger.critical("This is a CRITICAL message")

# Now raise the threshold to WARNING
print("\n--- After setting level to WARNING ---")
logger.setLevel(logging.WARNING)
logger.debug("This DEBUG will NOT appear")
logger.info("This INFO will NOT appear")
logger.warning("This WARNING will appear")
logger.error("This ERROR will appear")

# Clean up: remove handler to avoid duplicate output in later cells
logger.removeHandler(handler)
logger.setLevel(logging.NOTSET)

## Handlers: Directing Log Output

**Handlers** determine where log records go. A logger can have multiple handlers, each
directing records to a different destination with its own level threshold.

Common handler types:
- `StreamHandler` -- writes to `sys.stderr` (default) or any file-like object
- `FileHandler` -- writes to a file on disk
- `RotatingFileHandler` -- file handler with automatic rotation
- `SocketHandler`, `SMTPHandler`, `HTTPHandler` -- network-based handlers
- `NullHandler` -- discards all records (used in libraries)

In [ ]:
import logging
import sys

# Create a logger with a StreamHandler writing to stdout
logger = logging.getLogger("myapp.handlers")
logger.setLevel(logging.DEBUG)

# StreamHandler to stdout (default is stderr)
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.INFO)  # Only INFO and above on console

logger.addHandler(console_handler)

# DEBUG won't appear (below handler's threshold)
logger.debug("Debug details -- will not appear on console")

# INFO and above will appear
logger.info("Application started")
logger.warning("Disk space low")
logger.error("Connection failed")

# Each handler has its own level -- the logger level is the first gate,
# then each handler applies its own level filter
print(f"\nLogger level: {logging.getLevelName(logger.level)}")
print(f"Handler level: {logging.getLevelName(console_handler.level)}")

# Clean up
logger.removeHandler(console_handler)

## Formatters: Controlling Log Output Format

**Formatters** define the layout of each log message. You attach a formatter to a handler
using `handler.setFormatter(formatter)`.

Common format placeholders:

| Placeholder | Description |
|-------------|-------------|
| `%(levelname)s` | Level name (DEBUG, INFO, etc.) |
| `%(name)s` | Logger name |
| `%(message)s` | The log message |
| `%(asctime)s` | Timestamp |
| `%(filename)s` | Source file name |
| `%(lineno)d` | Line number |
| `%(funcName)s` | Function name |
| `%(module)s` | Module name |
| `%(pathname)s` | Full file path |
| `%(process)d` | Process ID |
| `%(thread)d` | Thread ID |

In [ ]:
import logging
import sys

logger = logging.getLogger("myapp.formatters")
logger.setLevel(logging.DEBUG)

# Create a handler with a detailed formatter
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)

# Format: timestamp - level - logger name - message
formatter = logging.Formatter(
    fmt="%(asctime)s | %(levelname)-8s | %(name)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
handler.setFormatter(formatter)
logger.addHandler(handler)

logger.debug("Initializing database connection")
logger.info("Server started on port 8080")
logger.warning("Cache miss for key 'user:42'")
logger.error("Failed to write to /var/log/app.log")

# Switch to a minimal formatter
print("\n--- Minimal format ---")
minimal_formatter = logging.Formatter("%(levelname)s: %(message)s")
handler.setFormatter(minimal_formatter)

logger.info("Same handler, different format")
logger.error("Errors are now more concise")

# Clean up
logger.removeHandler(handler)

## basicConfig(): Quick Setup

`logging.basicConfig()` provides a convenient one-call setup for the **root logger**.
It creates a `StreamHandler` with a default formatter and attaches it to the root logger.

**Important**: `basicConfig()` only works if the root logger has no handlers yet. Calling
it a second time has no effect unless you pass `force=True` (Python 3.8+).

In [ ]:
import logging

# Reset root logger for this demo
root = logging.getLogger()
for h in root.handlers[:]:
    root.removeHandler(h)

# basicConfig with common options
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    datefmt="%H:%M:%S",
    force=True,  # Override any existing configuration
)

# Now any logger will output through the root's handler
logging.debug("Root logger debug message")
logging.info("Root logger info message")
logging.warning("Root logger warning message")

# Named loggers also propagate to the root by default
app_logger = logging.getLogger("myapp")
app_logger.info("Message from myapp logger (propagated to root)")

# Clean up root logger
for h in root.handlers[:]:
    root.removeHandler(h)

## LogRecord Attributes and the Logging Flow

When you call `logger.info("message")`, the following sequence occurs:

1. The logger checks if the message's level meets the logger's threshold
2. A `LogRecord` object is created containing all metadata
3. Logger filters are applied (if any)
4. The record is passed to each handler on this logger
5. Each handler checks its own level threshold and filters
6. The handler's formatter converts the `LogRecord` to a string
7. The handler emits the formatted string to its destination
8. If `propagate=True` (default), the record is also passed to parent loggers

A `LogRecord` carries rich metadata that formatters and filters can use.

In [ ]:
import logging

# Create a LogRecord manually to inspect its attributes
record: logging.LogRecord = logging.LogRecord(
    name="myapp.module",
    level=logging.WARNING,
    pathname="/app/myapp/module.py",
    lineno=42,
    msg="Connection to %s timed out after %d seconds",
    args=("db.example.com", 30),
    exc_info=None,
)

# Key attributes on every LogRecord
attributes: list[str] = [
    "name", "levelname", "levelno", "pathname", "filename",
    "module", "lineno", "funcName", "created", "msecs",
    "relativeCreated", "thread", "threadName", "process",
]

print("LogRecord attributes:")
for attr in attributes:
    print(f"  {attr:>20}: {getattr(record, attr)}")

# The formatted message (msg % args)
print(f"\n  {'getMessage()':>20}: {record.getMessage()}")

In [ ]:
import logging
import sys


class InspectingHandler(logging.Handler):
    """A custom handler that prints LogRecord attributes for learning."""

    def emit(self, record: logging.LogRecord) -> None:
        """Print key attributes from the log record."""
        print(f"  LogRecord received:")
        print(f"    name      = {record.name}")
        print(f"    levelname = {record.levelname}")
        print(f"    message   = {record.getMessage()}")
        print(f"    created   = {record.created:.2f}")
        print(f"    thread    = {record.threadName}")
        print()


# Attach our inspecting handler to a logger
logger = logging.getLogger("myapp.inspect")
logger.setLevel(logging.DEBUG)
logger.propagate = False  # Don't send to root logger

inspector = InspectingHandler()
logger.addHandler(inspector)

# Each call creates and routes a LogRecord
logger.info("User %s logged in from %s", "alice", "192.168.1.10")
logger.error("Failed to process order #%d", 1234)

# Clean up
logger.removeHandler(inspector)
logger.propagate = True

## Practical: Setting Up Logging for an Application Module

Here is a complete example showing how to set up proper logging for an application
module. This pattern is the recommended approach for production code:

1. Each module creates its own logger with `logging.getLogger(__name__)`
2. The application entry point configures handlers and formatters
3. Loggers use lazy string formatting with `%s` placeholders (not f-strings)

In [ ]:
import logging
import sys


# --- Module code (e.g., myapp/payment.py) ---
# In a real module, this would be: logger = logging.getLogger(__name__)
payment_logger = logging.getLogger("myapp.payment")


def process_payment(user_id: int, amount: float, currency: str = "USD") -> bool:
    """Process a payment for a user."""
    payment_logger.info("Processing payment: user=%d, amount=%.2f %s", user_id, amount, currency)

    if amount <= 0:
        payment_logger.error("Invalid amount %.2f for user %d", amount, user_id)
        return False

    if amount > 10_000:
        payment_logger.warning("Large payment detected: %.2f %s for user %d", amount, currency, user_id)

    # Simulate processing
    payment_logger.debug("Connecting to payment gateway for user %d", user_id)
    payment_logger.debug("Payment authorized, transaction committed")
    payment_logger.info("Payment of %.2f %s completed for user %d", amount, currency, user_id)
    return True


# --- Application entry point (e.g., main.py) ---
def configure_logging(level: int = logging.DEBUG) -> None:
    """Configure application-wide logging."""
    root = logging.getLogger()
    root.setLevel(level)

    # Remove any existing handlers
    for h in root.handlers[:]:
        root.removeHandler(h)

    # Console handler with detailed format
    console = logging.StreamHandler(sys.stdout)
    console.setLevel(level)
    console.setFormatter(logging.Formatter(
        fmt="%(asctime)s [%(levelname)-8s] %(name)s: %(message)s",
        datefmt="%H:%M:%S"
    ))
    root.addHandler(console)


# Configure and run
configure_logging(logging.DEBUG)

app_logger = logging.getLogger("myapp")
app_logger.info("Application starting")

process_payment(user_id=42, amount=99.99)
process_payment(user_id=7, amount=-10.00)
process_payment(user_id=13, amount=15_000.00, currency="EUR")

app_logger.info("Application shutting down")

# Clean up root logger
for h in logging.getLogger().handlers[:]:
    logging.getLogger().removeHandler(h)

## Summary

### Key Takeaways

| Concept | Tool | Purpose |
|---------|------|--------|
| **Log levels** | `DEBUG` through `CRITICAL` | Classify message severity (10-50) |
| **Loggers** | `logging.getLogger(name)` | Create/retrieve named logger instances |
| **Handlers** | `StreamHandler`, `FileHandler` | Direct log output to destinations |
| **Formatters** | `logging.Formatter(fmt)` | Control the layout of log messages |
| **Quick setup** | `logging.basicConfig()` | One-call configuration for the root logger |
| **LogRecord** | Automatic | Carries all metadata through the logging pipeline |

### Best Practices
- Use `logging.getLogger(__name__)` in every module for automatic naming
- Use lazy formatting with `%s` placeholders, not f-strings, for log messages
- Configure handlers and formatters at the application level, not in libraries
- Set appropriate log levels: `DEBUG` for development, `INFO` or `WARNING` for production
- Always clean up handlers when reconfiguring to avoid duplicate output
- Prefer `logging` over `print()` for any diagnostic output that may need to persist